In [162]:
import json
import numpy
import requests
from bs4 import BeautifulSoup
from espncricinfo.exceptions import MatchNotFoundError, NoScorecardError
import pandas as pd

class Match(object):
    def __init__(self, match_id):
        self.match_id = match_id
        self.match_url = f"https://site.api.espn.com/apis/site/v2/sports/cricket/8048/summary?event={match_id}"
        self.json_url = f"https://www.espncricinfo.com/matches/engine/match/{match_id}.json"
        self.json = self.get_json()
        self.json_api = self.get_json_api()
        self.description = self.get_description()
        self.full_scorecard_url = self.get_full_scorecard_url()
        # self.teamNumbering = self.get_teamNumbering()
        self.toss = self.get_toss()
        self.home_team = self.get_home_team()
        self.away_team = self.get_away_team()
        self.batting_first = self.get_batting_first()
        self.batting_second = self.get_batting_second()
        # self.team = self.get_team()
        self.team1_batting_df = self.get_team1_batting_df()
        self.team2_batting_df = self.get_team2_batting_df()
        self.team1_bowling_df = self.get_team1_bowling_df()
        self.team2_bowling_df = self.get_team2_bowling_df()
        self.team1_dismissals_df = self.get_team1_dismissals_df()
        self.team2_dismissals_df = self.get_team2_dismissals_df()
        self.dream11_points = self.get_dream11_points()
        
    def get_json(self):
        r = requests.get(self.json_url)
        if r.status_code == 404:
            raise MatchNotFoundError
        elif 'Scorecard not yet available' in r.text:
            raise NoScorecardError
        else:
            return r.json()
    
    def get_json_api(self):
        r = requests.get(self.match_url)
        if r.status_code == 404:
            raise MatchNotFoundError
        elif 'Scorecard not yet available' in r.text:
            raise NoScorecardError
        else:
            return r.json()
        
    def get_full_scorecard_url(self):
        url  = self.description
        matchName = url.split(",")[1].split(":")[1].strip().split(" ")[:-2]
        firstHalf = '-'.join(matchName).lower().replace("-v-", "-vs-")
        matchNo = url.split(",")[1].split(":")[0].replace(" ","-").lower()
        finalurl = firstHalf + matchNo + "-" + str(self.match_id)
        return f"https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/{finalurl}/full-scorecard"
    
    def get_description(self):
        return self.json['description']
    
    def get_toss(self):
        return self.json_api['gameInfo']['venue']['fullName']

    
    def get_home_team(self):
        a = self.json_api['rosters'][0]['team']
        return {'name':a['abbreviation'], 'fullName' : a['displayName'], 'id' : a['id']}
    
    def get_away_team(self):
        a = self.json_api['rosters'][1]['team']
        return {'name':a['abbreviation'], 'fullName' : a['displayName'], 'id' : a['id']}
    
    # def get_team(self, team):
    #     team = []
    #     for i in self.json_api['rosters'][0]['roster']:
    #          team.append(i['athlete']['name'])
    #     return team
    
    def get_batting_first(self):
        a = self.json_api['matchcards'][0]
        teamName = a['teamName']
        otherTeamName = [self.home_team['name'],self.away_team['name']]
        otherTeamName.remove(teamName)
        inningsNumber = a['inningsNumber']
        headline = a['headline']
        if(headline == 'Batting'):
            if(inningsNumber == '1'):
                return teamName
            else:
                return otherTeamName[0]
        else:
            if(inningsNumber == '2'):
                return teamName
            else:
                return otherTeamName[0]

    def get_batting_second(self):
        a = self.json_api['matchcards'][0]
        teamName = a['teamName']
        otherTeamName = [self.home_team['name'],self.away_team['name']]
        otherTeamName.remove(teamName)
        inningsNumber = a['inningsNumber']
        headline = a['headline']
        if(headline == 'Batting'):
            if(inningsNumber == '1'):
                return otherTeamName[0]
            else:
                return teamName
        else:
            if(inningsNumber == '2'):
                return otherTeamName[0]
            else:
                return teamName

    def get_team1_batting_df(self):
        r = requests.get(self.full_scorecard_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        df = pd.DataFrame(columns= ['Name', "Wicket", "Runs", "Balls", "Fours", "Sixes"])
        tables = soup.find_all('table')
        brows = tables[0].findAll('tr')[1:-4]
        for brow in brows:
            x  = brow.findAll('td')
            if len(x) == 1:
                continue
            name = x[0].text
            name2 = name.replace('(c)','')
            name3 = name2.replace('†', '')
            name3 = name3.strip()
            df.loc[len(df)] = [name3, x[1].text, x[2].text, x[3].text, x[5].text, x[6].text]
        df[["Runs", "Balls", "Fours", "Sixes"]] = df[["Runs", "Balls", "Fours", "Sixes"]].astype(int)
        df["Strike Rate"] = (df["Runs"] / df["Balls"] * 100).round(2)
        return df

    def get_team2_batting_df(self):
        r = requests.get(self.full_scorecard_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        df = pd.DataFrame(columns= ['Name', "Wicket", "Runs", "Balls", "Fours", "Sixes"])
        tables = soup.find_all('table')
        brows = tables[2].findAll('tr')[1:-4]
        for brow in brows:
            x  = brow.findAll('td')
            if len(x) == 1:
                continue
            name = x[0].text
            name2 = name.replace('(c)','')
            name3 = name2.replace('†', '')
            name3 = name3.strip()
            df.loc[len(df)] = [name3, x[1].text, x[2].text, x[3].text, x[5].text, x[6].text]
        df[["Runs", "Balls", "Fours", "Sixes"]] = df[["Runs", "Balls", "Fours", "Sixes"]].astype(int)
        df["Strike Rate"] = (df["Runs"] / df["Balls"] * 100).round(2)
        return df
    
    def get_team1_bowling_df(self):
        table = pd.read_html(self.full_scorecard_url)
        df = table[1]
        df = df[~df['BOWLING'].str[0].str.isdigit()]
        df.rename(columns = {'O':'Overs', 'M' : 'Maidens', 'R' : 'Runs', 'W' : 'Wickets', 'ECON' : 'Economy'} , inplace= True)
        df[["Overs", "Economy",]] = df[["Overs","Economy"]].astype(float)
        df[["Maidens", "Runs", "Wickets", "0s", "4s", "6s", "WD", "NB"]] = df[["Maidens", "Runs", "Wickets", "0s", "4s", "6s", "WD", "NB"]].astype("int")
        return df
    
    def get_team2_bowling_df(self):
        table = pd.read_html(self.full_scorecard_url)
        df = table[3]
        df = df[~df['BOWLING'].str[0].str.isdigit()]
        df.rename(columns = {'O':'Overs', 'M' : 'Maidens', 'R' : 'Runs', 'W' : 'Wickets', 'ECON' : 'Economy'} , inplace= True)
        df[["Overs", "Economy",]] = df[["Overs","Economy"]].astype(float)
        df[["Maidens", "Runs", "Wickets", "0s", "4s", "6s", "WD", "NB"]] = df[["Maidens", "Runs", "Wickets", "0s", "4s", "6s", "WD", "NB"]].astype("int")
        return df
    
    def get_dream11_points(self):
        return None
    
    def get_team1_dismissals_df(self):
        df = pd.DataFrame(columns= ['Name', "Position", "Dismissal Type", "Bowler", "Catcher/Runout"])
        if self.home_team['name'] == self.batting_first:
            z = 0
        else:
            z = 1

        for i in self.json_api['rosters'][0]['roster']:
            name = i['athlete']['name']
            position = i['athlete']['position']['name']
            dismissalType = i['linescores'][z]['statistics']['categories'][0]['stats'][4]['value']
            try:
                if(len(i['linescores'][z]['statistics']['batting']['outDetails']['fielders']) > 1):
                    catcher = i['linescores'][z]['statistics']['batting']['outDetails']['fielders'][0]['athlete']['displayName'] + '/' + i['linescores'][0]['statistics']['batting']['outDetails']['fielders'][1]['athlete']['displayName']
                else:
                    catcher = i['linescores'][z]['statistics']['batting']['outDetails']['fielders'][0]['athlete']['displayName']
            except:
                catcher = None
            try:
                bowler = i['linescores'][z]['statistics']['batting']['outDetails']['bowler']['displayName']
            except:
                bowler = None
            df.loc[len(df)] = [name, position, dismissalType,bowler, catcher]
        # a = pd.read_json(m.match_url)
        return df
    
    def get_team2_dismissals_df(self):
        df = pd.DataFrame(columns= ['Name', "Position", "Dismissal Type", "Bowler", "Catcher/Runout"])

        if self.home_team['name'] == self.batting_first:
            z = 1
        else:
            z = 0


        for i in self.json_api['rosters'][1]['roster']:
            name = i['athlete']['name']
            position = i['athlete']['position']['name']
            dismissalType = i['linescores'][z]['statistics']['categories'][0]['stats'][4]['value']
            try:
                if(len(i['linescores'][z]['statistics']['batting']['outDetails']['fielders']) > 1):
                    catcher = i['linescores'][z]['statistics']['batting']['outDetails']['fielders'][0]['athlete']['displayName'] + '/' + i['linescores'][0]['statistics']['batting']['outDetails']['fielders'][1]['athlete']['displayName']
                else:
                    catcher = i['linescores'][z]['statistics']['batting']['outDetails']['fielders'][0]['athlete']['displayName']
            except:
                catcher = None
            try:
                bowler = i['linescores'][z]['statistics']['batting']['outDetails']['bowler']['displayName']
            except:
                bowler = None
            df.loc[len(df)] = [name, position, dismissalType,bowler, catcher]
        # a = pd.read_json(m.match_url)
        return df



In [163]:
matchNumber = 1426290
# t = Match(1426295)
m = Match(matchNumber)


/tmp/ipykernel_98437/4010464614.py:157: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'O':'Overs', 'M' : 'Maidens', 'R' : 'Runs', 'W' : 'Wickets', 'ECON' : 'Economy'} , inplace= True)
/tmp/ipykernel_98437/4010464614.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[["Overs", "Economy",]] = df[["Overs","Economy"]].astype(float)
/tmp/ipykernel_98437/4010464614.py:159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

In [164]:
x = m.team2_batting_df

x['Fantasy Points'] = x['Runs'] + x['Fours'] + x['Sixes']*2
x['Fantasy Points'] = x['Runs'] + x['Fours'] + x['Sixes']*2
x['Fantasy Points'] = numpy.where(x['Runs'] > 50, x['Fantasy Points'] + 8, x['Fantasy Points'])
x['Fantasy Points'] = numpy.where(x['Runs'] > 100, x['Fantasy Points'] + 8, x['Fantasy Points'])
x['Fantasy Points'] = numpy.where((x['Runs'] == 0) & (x['Wicket'] != 'not out '), x['Fantasy Points'] - 2 , x['Fantasy Points'])
x['Fantasy Points'] = numpy.where((x['Strike Rate'] <=70 ) & (x['Balls'] >= 10), x['Fantasy Points'] - 2 , x['Fantasy Points'])
x['Fantasy Points'] = numpy.where((x['Strike Rate'] <60) & (x['Balls'] >= 10), x['Fantasy Points'] - 2 , x['Fantasy Points'])
x['Fantasy Points'] = numpy.where((x['Strike Rate'] <50 ) & (x['Balls'] >= 10), x['Fantasy Points'] - 2 , x['Fantasy Points'])

y = m.team1_bowling_df

y['Fantasy Points'] = y['Wickets']*25 + y['Maidens']*8 
y['Fantasy Points'] = numpy.where(y['Wickets'] >= 4, y['Fantasy Points'] + 8, y['Fantasy Points'])
y['Fantasy Points'] = numpy.where(y['Wickets'] >= 5, y['Fantasy Points'] + 8, y['Fantasy Points'])
y['Fantasy Points'] = numpy.where((y['Economy'] <= 6) & (y['Overs'] >=2) , y['Fantasy Points'] +2, y['Fantasy Points'])
y['Fantasy Points'] = numpy.where((y['Economy'] <= 5) & (y['Overs'] >=2), y['Fantasy Points'] +2, y['Fantasy Points'])

